In [6]:
import numpy as np
from numpy.random import seed, choice
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from imblearn.metrics import specificity_score
import pandas as pd

# 1.Creación de la clase RandomForest para problemas de clasificación

Cada árbol (un total de n_estimators) se construye con un subconjunto del dataset con reemplazo (Bootstrap) y un subonjunto de variables. La variedad en las predicciones hace que RF sea más efectivo.

La técnica aplicada se conoce como Bagging = Bootstrap + Aggregating.

In [8]:
class RandomForestBootstrap:

    def __init__(self, n_estimators, random_state, max_depth, min_samples_leaf, max_features, X, y):
      self.n_estimators = n_estimators
      self.random_state = random_state
      self.max_depth = max_depth
      self.min_samples_leaf = min_samples_leaf
      self.max_features = max_features
      self.X = X
      self.y = y
      self.models = []

    def get_bootstrap_datasets(self):
        # Método para obtener conjuntos de datos bootstrap
        seed(self.random_state) # Se utiliza una semilla aleatoria para reproducir el experimento
        # Genera los índices bootstrap
        idxs = [choice(len(self.X), len(self.X), replace=True) for _ in range(self.n_estimators)]
        # Selección aleatoria de índices de características (max_features)
        feature_idxs = [choice(range(self.X.shape[1]), self.max_features, replace=False) for _ in range(self.n_estimators)]
        # Retorna los índices de las columnas y los datasets bootstrap como una lista de tuplas (X_bootstrap, y_bootstrap)
        return feature_idxs, [(self.X[idxs[i],:][:,feature_idxs[i]], self.y[idxs[i]]) for i in range(self.n_estimators)]

    def fit(self):
        # Método para ajustar el modelo (entrenar los árboles del bosque)
        feature_idxs, data_sets = self.get_bootstrap_datasets()
        for i, data in enumerate(data_sets):
          X, y = data
          # Si es clasificación, se entrena un árbol de clasificación
          self.models.append((feature_idxs[i], DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features, random_state=self.random_state).fit(X, y)))

    def predict(self, X):
      # Si hay modelos entrenados
      predictions = np.vstack([model.predict(X[:,idxs]) for idxs, model in self.models])
      # Se devuelve la clase más común entre las predicciones
      predicciones = [Counter(predictions[:,i]).most_common(1)[0][0] for i in range(predictions.shape[1])]
      return predicciones

# 2.Uso para una tarea de clasificación con un modelo simple y otro complejo

In [9]:
# Cargar el conjunto de datos de cáncer de mama
data = load_breast_cancer()
X = data.data
y = data.target

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Crear y entrenar un bosque simple
rf_simple = RandomForestBootstrap(n_estimators=5, max_depth=2, min_samples_leaf=20, max_features=X_train.shape[1]//2, random_state=42, X=X_train, y=y_train)
rf_simple.fit()

# Bosque complejo
rf_complex = RandomForestBootstrap(n_estimators=100, max_depth=20, min_samples_leaf=1, max_features=int(X_train.shape[1]*0.8), random_state=42, X=X_train, y=y_train)
rf_complex.fit()

In [ ]:
# Evaluar el modelo complejo en entrenamiento y prueba
train_pred_complex = rf_complex.predict(X_train)
test_pred_complex = rf_complex.predict(X_test)

# Evaluar el modelo simple en entrenamiento y prueba
train_pred_simple = rf_simple.predict(X_train)
test_pred_simple = rf_simple.predict(X_test)

In [ ]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [ ]:
# Métricas del modelo complejo
get_metrics(y_train, y_test, train_pred_complex, test_pred_complex)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000
Test,0.95614,0.965035,0.951032,0.958333,0.971831,0.930233
Diferencia,0.04386,0.034965,0.048968,0.041667,0.028169,0.069767


In [ ]:
# Métricas del modelo simple
get_metrics(y_train, y_test, train_pred_simple, test_pred_simple)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.956044,0.964664,0.956563,0.975000,0.954545,0.958580
Test,0.956140,0.965517,0.946446,0.945946,0.985915,0.906977
Diferencia,-0.000096,-0.000853,0.010117,0.029054,-0.031370,0.051603


El modelo simple muestra mucha menos varianza que el modelo complejo y además tiene un sesgo bajo ya que obtiene buenos resultados en ambos conjuntos. El modelo complejo memoriza y tiene alta varianza.

*Si cambias el muestreo (random_state en la función train_test_split) probablemente verás que el modelo simple con 2 estimadores funciona mejor sobre el conjunto de entrenamiento que sobre el conjunto de test, este mejor rendimiento en el conjuto de test puede ser debido al muestreo realizado, ya que a veces el conjunto de entrenamiento puede ser más difícil que el conjunto de prueba debido a la variabilidad en los datos de entrenamiento.